<a href="https://colab.research.google.com/github/cchen744/uhi-extreme-heat-response/blob/cell-delta-uhi-2/notebooks/03_uhi_n_landcover.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ΔUHI & Built-environment
1. **Goal**: To investigate the possible relationship between SUHI response and built environment. Since we observed a significant increase in SUHI in several cities while others not, we are assuming that if this condition-dependence might be contributed to by composition of the built environment; we test whether this is true.

2. **Outcome Variable: Defining SUHI condition-dependence**: in this step, we define SUHI's condition-dependence as:
    
    *ΔUHI = UHI_extreme − UHI_baseline*

    - UHI_extreme: SUHI when the daily mean temperature is over 90 percentile of the daily mean temperature
    - UHI_baseline: the average of SUHI when the daily mean temperature is between 50-70 percentile.

    (Percentiles are computed within the warm-season window to ensure comparability across cities.)

3. **Explanatory Variables: Built-environment characteristics**:

    **Surface composition**:
      - Impervious surface fraction
      - Vegetation / NDVI / tree cover
      - Water or bare land fraction
      - LCZ composition

    **Urban form & intensity proxies**:
      - Built-up density / road density

4. **Analytical Strategy**
  - Analytical Unit: grid cell within each city (resolution =
  - Model:
  
    *ΔUHI_cell ~ composition_cell + proxies_cell + city fixed effects*

  - Comparison logic:
    - within-city: which built factors is correlated with ΔUHI_cell
    - across-city: does this explain why some city has higher ΔUHI
  
  - Control principles:
    - Same buffer scale
    - Same spatial resolution
    - Same seasonal window



In [1]:
!git init
!git remote add origin https://github.com/cchen744/uhi-extreme-heat-response.git
!git pull origin cell-delta-uhi --allow-unrelated-histories
!git checkout cell-delta-uhi
!git status

!git config --global user.email cchen744@wisc.edu
!git config --global user.name cchen744

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/
remote: Enumerating objects: 317, done.
remote: Counting objects: 100% (158/158), done.
remote: Compressing objects: 100% (113/113), done.
remote: Total 317 (delta 88), reused 82 (delta 40), pack-reused 159 (from 2)
Receiving objects: 100% (317/317), 10.59 MiB | 7.33 MiB/s, done.
Resolving deltas: 100% (149/149), done.
From https://github.com/cchen744/uhi-extreme-heat-response
 * branch            cell-delta-uhi -> FETCH_HEAD
 * [new branch]      cell-delta-uh

In [2]:
from pathlib import Path
import os
import pandas as pd
import ee
import uhi_pipeline
import importlib
# importlib.reload(uhi_pipeline)
# print("uhi_pipeline module reloaded.")
from pprint import pprint

ee.Authenticate()
ee.Initialize(project='extremeweatheruhi')

DATA_DIR = Path("data/city_cell")
DATA_DIR.mkdir(parents=True, exist_ok=True)

ua_fc = ee.FeatureCollection("projects/extremeweatheruhi/assets/uac20_2025")

In [11]:
importlib.reload(uhi_pipeline)
print("uhi_pipeline module reloaded.")

uhi_pipeline module reloaded.


Before starting analysis, we need to get daily output on the level of grid cell due to the need for intra-city built environment analysis. In uhi_pipeline.py, the grid cell is defined as a 1km x 1km square grid projected from EPSG: 3875.

In [4]:
city_fc=uhi_pipeline.select_ua(ua_fc,ua_contains="Phoenix")
city_geom = city_fc.geometry()
ic = (ee.ImageCollection("MODIS/061/MYD11A1")
      .filterBounds(city_geom)
      .filterDate("2013-07-01", "2013-08-01"))
print("IC count:", ic.size().getInfo())

IC count: 31


In [5]:
lcz_img = ee.ImageCollection("RUB/RUBCLIM/LCZ/global_lcz_map/latest").first()
lcz = lcz_img.select("LCZ_Filter")
lst_scale_m = 1000
ring_outer_m= 12000
ring_inner_m= 3000
agg_func="median"
cell_crs="EPSG:3857"

start_date="2013-07-01"
end_date="2013-08-01"
unit="cell", # modify unit to 'cell'
cell_scale_m=500
lst_band="LST_Night_1km"
qc_band="QC_Night"

BUILT_MIN, BUILT_MAX = 1, 10
WATER_CODE = 17

is_built = lcz.gte(BUILT_MIN).And(lcz.lte(BUILT_MAX))
is_water = lcz.eq(WATER_CODE)
is_natural = is_built.Not().And(is_water.Not())

urban_region = city_geom
outer = city_geom.buffer(ring_outer_m)
inner = city_geom.buffer(ring_inner_m)
rural_region = outer.difference(inner)

urban_mask = is_water.Not().clip(urban_region)
rural_mask = is_natural.clip(rural_region)

In [7]:
# Test new uhi_pipeline.py with new core function make_monthly_table_cells()
df = uhi_pipeline.run_city(
    ua_fc=ua_fc,
    ua_contains="Phoenix",
    start_date="2019-06-01",
    end_date="2019-08-01",
    cell_scale_m=500,
    cell_crs="EPSG:3857",
    lst_band="LST_Night_1km",
    qc_band="QC_Night",
    lst_scale_m=1000,
    min_cell_pixels=50,
    min_rural_pixels=50,
    export_to_drive=False,
    debug=True,
    unit='cell',
)

print(df.head(10))
print(df.shape)
print(df.dtypes)

fc_list length: 1


EEException: Computation timed out.

In [12]:
# test partial logic of run_city function within uhi_pipeline
city_fc = uhi_pipeline.select_ua(ua_fc, ua_contains="Phoenix")
city_geom = city_fc.geometry()

urban_region, rural_region, urban_mask, rural_mask = uhi_pipeline.build_masks(
    city_geom, 12000, 3000
)

# execute make_monthly_table_cells to see if there is any output
fc = uhi_pipeline.make_monthly_table_cells(
    "2019-06-01", "2019-08-01",
    urban_region, rural_region,
    urban_mask, rural_mask,
    "LST_Night_1km", "QC_Night",
    lst_scale_m=1000,
    cell_scale_m=1000,   # use 1000m first to avoid overload of grids
    crs="EPSG:3857"
)

print(type(fc))
print(fc.size().getInfo())
print(fc.first().toDictionary().getInfo())

<class 'ee.featurecollection.FeatureCollection'>
9254
{'LST_rur': 21.443089332113676, 'LST_urb_cell': 22.443846153846188, 'cell_id': -1240899996051, 'cell_n': 1, 'count': 1, 'delta_uhi': 1.0007568217325122, 'mean': 22.443846153846188, 'month': '2019-06', 'rural_n': 4644}


In [ ]:
print(fc.first().propertyNames().getInfo())

['system:index', 'month', 'LST_rur', 'rural_n', 'cell_id']


In [ ]:
# to be continued
print(fc.aggregate_histogram("month").getInfo())

{'2019-06': 4627, '2019-07': 4627}


In [13]:
commit_msg = 'UPDATE: merged cell table generation logic with city table generation logic'
! git add uhi_pipeline.py
! git commit -m commit_msg
! git push --set-upstream origin cell-delta-uhi-2

[cell-delta-uhi-2 2fd8208] commit_msg
 1 file changed, 89 insertions(+), 171 deletions(-)
fatal: could not read Username for 'https://github.com': No such device or address
